In [10]:
import os
import json
import datetime
from dateutil import parser
import glob
import json
from dataclasses import dataclass, field
from dacite import from_dict, Config

CREDENTIALS_FILE = "./creds.txt"
USERDIR = "datasets/users"
QUERY_RESULTS = "queried_users.jsonl"
TWEET_QUERY_RESULTS = "queried_users_tweets.jsonl"

# TT_hongkong.nd json
# First Tweet retrieved Friday, August 9, 2019 8:33:46 PM EST
# Result_type: recent

In [3]:
@dataclass
class Tweet:
    id: int
    text: str
    created_at: str
    lang: str
    source: str
    retweeted: bool

@dataclass
class User:
    id: int
    screen_name: str
    name: str
    description: str
    location: str
    tweets: list[Tweet] = field(default_factory=list)

In [9]:
users = {}
with open(QUERY_RESULTS, "r") as f:
    for line in f:
        line = line.strip()
        data = json.loads(line)
        users[data["id"]] = data
        
users_control = {}
with open("control_" + QUERY_RESULTS, "r") as f:
    for line in f:
        line = line.strip()
        data = json.loads(line)
        users_control[data["id"]] = data

print("% Accounts that exist still today")
print("EXPERIMENT:")
print(sum([result["found"] and not result["protected"] for result in users.values()])/float(len(users)))
print("CONTROL:")
print(sum([result["found"] and not result["protected"] for result in users_control.values()])/float(len(users_control)))

% Accounts that exist still today
EXPERIMENT:
0.8448712536935415
CONTROL:
0.8177033492822966


In [17]:
tweets = {}
rts = 0
with open(TWEET_QUERY_RESULTS, "r") as f:
    for line in f:
        line = line.strip()
        data = json.loads(line)
        if data["found"] and data["text"].startswith("RT @"):
            rts += 1
            continue
        tweets[data["id"]] = data
        
rts_control = 0
tweets_control = {}
with open("control_" + TWEET_QUERY_RESULTS, "r") as f:
    for line in f:
        line = line.strip()
        data = json.loads(line)
        if data["found"] and data["text"].startswith("RT @"):
            rts_control += 1
            continue
        tweets_control[data["id"]] = data

# TODO: CONTROL FOR DELETED RTS
print("% Tweets that exist still today")
print("EXPERIMENT:")
print(sum([result["found"] for result in users.values()])/float(len(users)))
print("CONTROL:")
print(sum([result["found"] for result in users_control.values()])/float(len(users_control)))

print("==============")
print("% RTs")
print("EXPERIMENT:")
print(rts / float(rts + len(tweets)))
print("CONTROL:")
print(rts_control / float(rts_control + len(tweets)))

% Tweets that exist still today
EXPERIMENT:
0.8997400064998375
CONTROL:
0.7321307011572499
% RTs
EXPERIMENT:
0.2951251218719532
CONTROL:
0.04667926684541339
